# Setup cell

In [1]:
import cv2 as cv
from cv2 import aruco
import numpy as np

from util import imagesToHtmlPage, get_calibration

def reset():
    cv.destroyAllWindows()
    try:
        cap.release()
    except:
        pass

reset()
print(cv.__version__)

3.2.0


# Potential approaches

## Markers in space, camera on robot
- diamond markers so orientation can be calculated too
- calculate angles by only moving one motor at a time
- limit kill-switches still needed
- camera could be mounted to end of robot ("hand") or middle ("forearm") of robot

## Markers on robot, camera fixed
- facing robot side-on
 - potentially limits potential yaw as markers must all remain visible

- Overhead view
 - aids with drawing tasks

## Markers on robot, camera on robot
- camera looking down/up arm
- calculate positions of markers
- could limit pitches of motors, depending on camera angle 

# Camera calibration

Only needed one time per camera

use `./aruco/create_board_charuco` to generate a board and `./aruco/calibrate_camera_charuco` to calibrate the camera (outputs a yaml file)

load the yaml file using `get_calibration(filename)`

# obselete

In [22]:
_blockSize=widgets.IntSlider(min=1, max=10, value=2)
_ksize=widgets.IntSlider(min=1, max=10, value=3)
_k=widgets.FloatSlider(min=0.0, max=0.1, value=0.04, step=0.005)
display(_blockSize, _ksize, _k)

IntSlider(value=2, max=10, min=1)

IntSlider(value=3, max=10, min=1)

FloatSlider(value=0.04, max=0.1, step=0.005)

(edit sliders then run next cell)

In [24]:
blockSize, ksize, k = (_blockSize.value, _ksize.value, _k.value)
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # Display the resulting frame
    gray = np.float32(gray)

    dst = cv.cornerHarris(gray, blockSize, ksize, k)
    #result is dilated for marking the corners, not important
    dst = cv.dilate(dst,None)
    # Threshold for an optimal value, it may vary depending on the image.
    frame[dst>0.5*dst.max()]=[0,0,255]
#     ret, dst = cv.threshold(dst,0.01*dst.max(),255,0)
    cv.imshow('frame', dst)
    if cv.waitKey(1) == ord('q'):
        break
cv.destroyAllWindows()
cap.release()

# Aruco detection
Tutorial: https://docs.opencv.org/3.1.0/d5/dae/tutorial_aruco_detection.html

Docs: https://docs.opencv.org/3.1.0/d4/d17/namespacecv_1_1aruco.html

## Single marker generation

In [7]:
n = 12

for i in range(n):
    marker = aruco.drawMarker(aruco.Dictionary_get(aruco.DICT_4X4_100),i,6)
    cv.imwrite(f'./out/marker_{i}.jpg',marker)
imagesToHtmlPage('./out/all.html', [f'marker_{i}.jpg' for i in range(n)], image_width="2.4cm", image_margins="2cm")

In [97]:
from IPython.display import Image, HTML
display(HTML('<style>.jp-RenderedImage.jp-OutputArea-output img {image-rendering: crisp-edges;}</style>'))
display(Image(filename='./markers/marker_0.jpg', height=100, width=100))

# ChArUco diamond markers

(more accurate pose detection than single markers)

generation code not yet ported (conversion from/to Vec4i is not wrapped in core)?

https://github.com/opencv/opencv_contrib/tree/master/modules/aruco/samples

https://github.com/opencv/opencv_contrib/blob/e8916dae98e13c080166156ec511c0c2d9197f60/modules/aruco/include/opencv2/aruco/charuco.hpp

In [27]:
# %mkdir ./diamond/out
import subprocess
# data = [
#     [0,0,0,1],
#     [0,0,0,2]
# ]
data = [[0,0,0,a] for a in range(4)]
for ids in data:
    id_strings = [str(a) for a in ids]
    subprocess.run(["./diamond/GenerateDiamond"] + id_strings + [f'./diamond/out/{"_".join(id_strings)}.jpg'])

mkdir: cannot create directory ‘./diamond/out’: File exists
0_0_0_0.jpg  0_0_0_1.jpg  0_0_0_2.jpg  0_0_0_3.jpg  0_1_2_4.jpg  1_1_3_4.jpg


In [30]:
filenames = [f'./{"_".join([str(a) for a in ids])}.jpg' for ids in data]
print(filenames)
imagesToHtmlPage('./diamond/out/out.html',filenames, image_width='10cm')

['./0_0_0_0.jpg', './0_0_0_1.jpg', './0_0_0_2.jpg', './0_0_0_3.jpg']


## Diamond detection

### From image

In [8]:
reset()
img = cv.imread('')

corners, ids, rejectedImgPoints = aruco.detectMarkers(img, aruco.Dictionary_get(aruco.DICT_6X6_250))
diamondCorners, diamondIds = aruco.detectCharucoDiamond(img, corners, ids, 200/120)
aruco.drawDetectedDiamonds(img, diamondCorners, diamondIds)

cv.imshow('img', img)
cv.waitKey(500)

### From webcam

In [9]:
reset()
DICT_PRESET = aruco.DICT_4X4_50
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
#     frame = cv.undistort(frame, camera_matrix, distortion_coefficients)
    # undistort
    camera_matrix, distortion_coefficients, newcameramtx, roi = get_calibration('./aruco/webcam.yaml')
    frame = cv.undistort(frame, camera_matrix, distortion_coefficients, None, newcameramtx)
    x, y, w, h = roi
    frame = frame[y:y+h, x:x+w]
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
#     frame=gray
    # Display the resulting frame
#     gray = np.float32(gray)

    corners, ids, rejectedImgPoints = aruco.detectMarkers(frame, aruco.Dictionary_get(DICT_PRESET))
    if ids is not None:
        diamondCorners, diamondIds = aruco.detectCharucoDiamond(frame, corners, ids, 200/120)
        if diamondIds is not None:
            frame = aruco.drawDetectedDiamonds(frame, diamondCorners, diamondIds)
            if len(diamondIds)>1:
                #print(aruco.estimatePoseSingleMarkers(diamondCorners, 200, 120, camera_matrix))
                #rvecs, tvecs = aruco.estimatePoseSingleMarkers(diamondCorners, 0.024, camera_matrix, distortion_coefficients)
                rvecs, tvecs = aruco.estimatePoseSingleMarkers(diamondCorners, 0.04, camera_matrix, distortion_coefficients)

#                 print(tvecs)
                cv.putText(frame,str(round(np.linalg.norm(tvecs[0]-tvecs[1]),5)),(0,30),cv.FONT_HERSHEY_SIMPLEX,1,(255,255,255))
#                 print(np.linalg.norm(tvecs[0]-tvecs[1]))
    #dst = cv.cornerHarris(gray, blockSize, ksize, k)
    #result is dilated for marking the corners, not important
    #dst = cv.dilate(dst,None)
    # Threshold for an optimal value, it may vary depending on the image.
    #frame[dst>0.5*dst.max()]=[0,0,255]
    #ret, dst = cv.threshold(dst,0.01*dst.max(),255,0)
    cv.imshow('frame', frame)
    if cv.waitKey(1) == ord('q'):
        break
cv.destroyAllWindows()
cap.release()

In [35]:
diamondCorners, diamondIds

([array([[[ 231.71800232,  335.96038818]],
  
         [[ 283.67782593,  336.72061157]],
  
         [[ 283.32400513,  389.6307373 ]],
  
         [[ 230.66191101,  388.92572021]]], dtype=float32),
  array([[[ 234.50323486,  140.99964905]],
  
         [[ 286.02099609,  141.01049805]],
  
         [[ 285.29620361,  192.39378357]],
  
         [[ 233.72932434,  192.20053101]]], dtype=float32)],
 array([[[0, 0, 0, 2]],
 
        [[0, 0, 0, 1]]], dtype=int32))

In [6]:
print(corners, ids)

[array([[[ 240.,  440.],
        [ 359.,  440.],
        [ 359.,  559.],
        [ 240.,  559.]]], dtype=float32), array([[[ 440.,  240.],
        [ 559.,  240.],
        [ 559.,  359.],
        [ 440.,  359.]]], dtype=float32), array([[[  40.,  240.],
        [ 159.,  240.],
        [ 159.,  359.],
        [  40.,  359.]]], dtype=float32), array([[[ 240.,   40.],
        [ 359.,   40.],
        [ 359.,  159.],
        [ 240.,  159.]]], dtype=float32)] [[3]
 [2]
 [1]
 [0]]


# Diamond webcam detection bak

In [10]:
                diamondCorners, diamondIds = aruco.detectCharucoDiamond(image, corners, ids, 200/120)
                if diamondIds is not None:
                    image = aruco.drawDetectedDiamonds(image, diamondCorners, diamondIds)
                    if len(diamondIds)>1:
                        print('====')
                        rvecs, tvecs = aruco.estimatePoseSingleMarkers(diamondCorners, 0.04, camera_matrix, distortion_coefficients)

                        tvecs_new = tvecs.reshape((-1,3))
                        diamondIds_new = diamondIds.reshape((-1,4))

                        # diamondIds.reshape((-1,4))
                        # print(diamondIds)
                        def find_marker_index(diamondId):
                            indexes = np.where(np.all(diamondIds_new == diamondId, axis=1))[0]
                            return indexes[0] if len(indexes > 0) else None

                        index_of_base = find_marker_index((0,0,0,0))
                        
                        print('b', diamondIds_new[index_of_base], tvecs_new[index_of_base])
                        #print(aruco.estimatePoseSingleMarkers(diamondCorners, 200, 120, camera_matrix))
                        #rvecs, tvecs = aruco.estimatePoseSingleMarkers(diamondCorners, 0.024, camera_matrix, distortion_coefficients)
                        
                        print('a', tvecs_new[index_of_base,1])
                        # print(tvecs)
                        print(np.linalg.norm(tvecs[0]-tvecs[1]))
                        print(str(round(np.linalg.norm(tvecs[0]-tvecs[1]),5)))
                        print('rvecs', rvecs)
                        print('tvecs', tvecs_new)
                        print('====')
                        cv.putText(image,str(round(np.linalg.norm(tvecs[0]-tvecs[1]),5)),(0,30),cv.FONT_HERSHEY_SIMPLEX,1,(255,255,255))
                        # print(np.linalg.norm(tvecs[0]-tvecs[1]))